# Install dependencies

In [8]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /private/var/folders/_3/b70qb3856jv2v4tk1htvvt1w0000gn/T/pip-req-build-mms3c4yp
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /private/var/folders/_3/b70qb3856jv2v4tk1htvvt1w0000gn/T/pip-req-build-mms3c4yp
^C
ERROR: Operation cancelled by user


In [9]:
!pip install qwen-vl-utils

In [10]:
!pip install torchvision

In [11]:
!pip install accelerate

In [1]:
import torch
import os
import json
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

In [13]:
if torch.backends.mps.is_available():
    device = "mps"
    print("Using MPS (Metal Performance Shaders) on Mac")
elif torch.cuda.is_available():
    device = "cuda"
    print("Using CUDA (GPU)")
else:
    device = "cpu"
    print("Using CPU")

Using MPS (Metal Performance Shaders) on Mac


In [14]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
)

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

In [15]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [16]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "file:///path/to/your/image.jpg"},
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

In [18]:
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("mps")

generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

RuntimeError: MPS backend out of memory (MPS allocated: 12.19 GB, other allocations: 2.78 MB, max allowed: 18.13 GB). Tried to allocate 6.10 GB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
# Function to perform inference on images
def process_images(data_directory, prompt="Detect drones in the image. If a drone is detected, return only the bounding box coordinates in the format (x, y, w, h) where (x, y) is the top-left corner of the bounding box and w and h are the width and height, no other text or information, only the coordinates."):
    results_dict = {}

    for filename in os.listdir(data_directory):
        if filename.endswith('.jpg'):
            image_path = os.path.join(data_directory, filename)
            messages = [
                {
                    "role": "user",
                    "content": [
                        {"type": "image", "image": f"file://{image_path}"},
                        {"type": "text", "text": prompt},
                    ],
                }
            ]
            text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            image_inputs, video_inputs = process_vision_info(messages)
            inputs = processor(
                text=[text],
                images=image_inputs,
                videos=video_inputs,
                padding=True,
                return_tensors="pt",
            )
            inputs = inputs.to(device)
            generated_ids = model.generate(**inputs, max_new_tokens=128)
            generated_ids_trimmed = [
                out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
            ]
            output_text = processor.batch_decode(
                generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
            )[0]

            results_dict[filename] = {
                "Inference": output_text
            }

    output_path = os.path.join(data_directory, 'output/inference_results.json')
    with open(output_path, 'w') as json_file:
        json.dump(results_dict, json_file, indent=4)

    print(f"Results saved to {output_path}")


process_images('path_to_your_data_folder')

In [ ]:
# # Install dependencies
!pip install git+https://github.com/huggingface/transformers
!pip install qwen-vl-utils
!pip install torchvision
!pip install accelerate
import torch
import os
import json
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

if torch.backends.mps.is_available():
    device = "mps"
    print("Using MPS (Metal Performance Shaders) on Mac")
elif torch.cuda.is_available():
    device = "cuda"
    print("Using CUDA (GPU)")
else:
    device = "cpu"
    print("Using CPU")

model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

# Function to perform inference on images
def process_images(data_directory, prompt="Detect drones in the image. If a drone is detected, return only the bounding box coordinates in the format (x, y, w, h) where (x, y) is the top-left corner of the bounding box and w and h are the width and height, no other text or information, only the coordinates"):
    results_dict = {}

    # Loop through all image files in the directory
    for filename in os.listdir(data_directory):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            # Process the image
            image_path = os.path.join(data_directory, filename)
            messages = [
                {
                    "role": "user",
                    "content": [
                        {"type": "image", "image": f"file://{image_path}"},
                        {"type": "text", "text": prompt},
                    ],
                }
            ]
            text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            image_inputs, video_inputs = process_vision_info(messages)
            inputs = processor(
                text=[text],
                images=image_inputs,
                videos=video_inputs,
                padding=True,
                return_tensors="pt",
            )
            inputs = inputs.to(device)
            generated_ids = model.generate(**inputs, max_new_tokens=128)
            generated_ids_trimmed = [
                out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
            ]
            output_text = processor.batch_decode(
                generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
            )[0]

            # Save results (inference) for the image
            results_dict[filename] = {
                "Inference": output_text
            }

    # Save the results to a JSON file
    output_path = os.path.join(data_directory, 'output/inference_results.json')
    with open(output_path, 'w') as json_file:
        json.dump(results_dict, json_file, indent=4)

    print(f"Results saved to {output_path}")


# Replace 'path_to_your_data_folder' with the actual path
process_images('images/')